## RESPUESTAS 
#### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?

Sobre las $k$ clases.

2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

`tensor_inv_covs` es un array de NumPy de $\mathbb{R}^{k \times p \times p}$

`tensor_means` es un array de NumPy de $\mathbb{R}^{k \times p \times 1}$

QDA en el método `_predict_one` itera sobre las $k$ clases calculando el logaritmo de la probabilidad a posteriori para cada una, usando el método `_predict_log_conditional` (este método devuelve un escalar), obteniendo una lista (de tamaño $k$) con la probabilidad de que la observación pertenezca a cada una de las clases. Luego, devuelve el argumento de la mayor probabilidad en dicha lista.

- inv_cov $\in \mathbb{R}^{p \times p}$
- means, unbiased_x $\in \mathbb{R}^{p \times 1}$
- unbiased_x.T @ inv_cov @ unbiased_x $\rarr$ $(1 \times p) @ (p \times p) @ (p \times 1) $ $\in \mathbb{R}^{1 \times 1} \rarr $ escalar

Entonces, `_predict_log_conditional` devuelve un escalar.

TensorizedQDA directamente hace el cálculo de los logaritmos de las probabilidades a posteriori de forma matricial en el método `_predict_log_conditionals`, el cual directamente devuelve un vector columna $\mathbb{R}^{k \times 1}$, similar a la lista obtenida con el bucle for dentro del método `_predict_one` de la clase `QDA`. Luego, el método `_predict_one` para esta clase solo se encarga de encotrar el argumento que maximiza la probabilidad (logarítmica).

- tensor_inv_cov $\in \mathbb{R}^{k \times p \times p}$
- tensor_means, unbiased_x  $\in \mathbb{R}^{k \times p \times 1}$
- inner_prod $\in \mathbb{R}^{k \times 1}$

--------------------------------------------------------------

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.

### Objetivo
Poder eliminar el ciclo for de predict. Es decir, predecir las n observaciones y k clases en un mismo paso. 

En BaseBayesianClassifier

predict(self, X) → tiene ciclo for recorre las n observaciones, y llama a predict_one, donde TensorizedQDA ya paraleliza las k clases. 

-------------
def predict(self, X): → llama a predict_one por cada observación

def _predict_one(self, x): → llama a _predict_log_conditional para hacer el argmax de la suma

$$
\log\hat{f}_j(x) + \log\hat{\pi}_j
$$

def _predict_log_conditional(self, x, class_idx)
$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$
----------------

Buscamos calcular la forma cuadrática para muchas observaciones a la vez:

$$(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$$

Donde usamos : _predict_log_conditional(x, class_idx)

unbiased_x = x - mean_j

unbiased_x.T @ inv_cov_j @ unbiased_x

## Librerías

In [1]:
# imports
import numpy        as np
import pandas       as pd
import numpy.linalg as LA
from scipy.linalg           import cholesky, solve_triangular
from scipy.linalg.lapack    import dtrtri

from base.qda               import QDA, TensorizedQDA
from base.cholesky          import QDA_Chol1, QDA_Chol2, QDA_Chol3
from utils.bench            import Benchmark
from utils.datasets         import (get_letters_dataset,label_encode)                                                     
from numpy.random           import RandomState

### Dataset 

In [2]:
# dataset de letters
X_letter, y_letter = get_letters_dataset()

# encoding de labels
y_letter_encoded = label_encode(y_letter.reshape(-1,1)) # hago reshape para que quede como matriz columna

# instanciacion del benchmark
b = Benchmark(
    X_letter, y_letter_encoded,
    same_splits=False,
    n_runs=100,
    warmup=20,
    mem_runs=30,
    test_sz=0.2
)

Benching params:
Total runs: 150
Warmup runs: 20
Peak Memory usage runs: 30
Running time runs: 100
Train size rows (approx): 16000
Test size rows (approx): 4000
Test size fraction: 0.2


### Prueba QDA

In [3]:
qda = QDA()

qda.fit(X_letter.T, y_letter_encoded)

qda.predict(X_letter.T[:, :5])

array([[25,  5, 18,  7,  7]])

### Prueba TensorizedQDA

In [4]:
tqda = TensorizedQDA()

tqda.fit(X_letter.T, y_letter_encoded)

tqda.predict(X_letter.T[:, :5])

array([[25,  5, 18,  7,  7]])

In [5]:
# QDA 
b.bench(QDA)

# TensorizedQDA
b.bench(TensorizedQDA)


QDA (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

QDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

TensorizedQDA (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

TensorizedQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

### FasterQDA

Definir una clase FasterQDA que herede de TensorizedQDA y redefina predict para predecir todas las observaciones juntas, sin el for sobre filas de X

In [6]:
class FasterQDA(TensorizedQDA):
    """
    Versión vectorizada (sin bucles en predict), 
    con matriz intermedia ineficiente (matriz N x N).
    """

    def _predict_log_conditionals_batch(self, X):
        # Dimensiones iniciales
        k, p, _ = self.tensor_means.shape # self.tensor_means: (k, p, 1)    → k clases, p features
        n = X.shape[1]                    # X: (p, n)                       → p features, n observaciones

        # Cálculo resta: D = X - medias
        D = X[None, :, :] - self.tensor_means   # X: (1, p, n) - Medias: (k, p, 1) → D: (k, p, n)

        # Cálculo de la distancia cuadrática 
        quad_terms = np.empty((k, n)) # Almacena distancias por cada k clase y observación n

        for j in range(k):
            D_j = D[j]                          # Shape: (p, n)
            inv_cov_j = self.tensor_inv_cov[j]  # Shape: (p, p)

            # ---------------------------------------------------------
            # Generación de matriz n x n
            # ---------------------------------------------------------
            # Al multiplicar D_j.T (n, p) @ inv (p, p) @ D_j (p, n), el resultado final es una matriz de (n, n)
            # solo nos importa cada dato consigo mismo (la diagonal)
            
            Q_j = D_j.T @ inv_cov_j @ D_j      # matriz (n, n)

            # Nos quedamos con la diagonal 
            quad_terms[j, :] = np.diag(Q_j)    
            # ---------------------------------------------------------

        
        # log_det es (k,). Lo convertimos a (k, 1) para operar con quad_terms
        log_det = np.log(LA.det(self.tensor_inv_cov))[:, None]
        
        # Fórmula final: 0.5 * log_det - 0.5 * distancia
        return 0.5 * log_det - 0.5 * quad_terms

    def predict(self, X):
        # traigo el log-condicionales 
        log_cond = self._predict_log_conditionals_batch(X)

        # Sumamos el log_a_priori 
        log_post = self.log_a_priori[:, None] + log_cond

        # Elegimos la clase ganadora → por columna
        y_hat = np.argmax(log_post, axis=0)

        return y_hat.reshape(1, -1)

----
5. Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$
es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,
$$
np.sum(A^T \odot B, axis=0).T
$$
queda a preferencia del alumno cuál usar.

### Demostración paso a paso

El elemento $(i,j)$ de $A \cdot B$ se calcula como:
$$
(A \cdot B)_{ij} = \sum_{k=1}^{p} A_{ik} \cdot B_{kj}
$$

Es decir, el **producto interno** entre la fila $i$ de $A$ y la columna $j$ de $B$.

Para la diagonal, solo nos interesan los elementos donde $i = j$:
$$
(A \cdot B)_{ii} = \sum_{k=1}^{p} A_{ik} \cdot B_{ki}
$$

Se observa que $B_{ki}$ es el elemento en la posición $(k, i)$ de $B$, que es exactamente el elemento $(i, k)$ de $B^T$.

Por lo tanto:
$$
(A \cdot B)_{ii} = \sum_{k=1}^{p} A_{ik} \cdot (B^T)_{ik}
$$


Si hacemos el producto elemento a elemento (Hadamard) $A \odot B^T$ :
$$
(A \odot B^T)_{ik} = A_{ik} \cdot (B^T)_{ik}
$$

Y luego sumamos a lo largo de las columnas (axis=1):
$$
\sum_{k=1}^{p} (A \odot B^T)_{ik} = \sum_{k=1}^{p}  A_{ik} \cdot (B^T)_{ik}    = (A \cdot B)_{ii}
$$

### EJEMPLO matriz 2x2

Haciendo el producto matricial entre $A$ y $B$:
$$
A = \begin{pmatrix}
a & b \\
c & d
\end{pmatrix}
\quad
B = \begin{pmatrix}
e & f \\
g & h
\end{pmatrix}
$$

$$
A \cdot B = \begin{pmatrix}
a & b \\
c & d
\end{pmatrix}
\begin{pmatrix}
e & f \\
g & h
\end{pmatrix}
$$

$$
A \cdot B = \begin{pmatrix}
ae + bg & af + bh \\
ce + dg & cf + dh
\end{pmatrix}
$$

$$
\text{diag}(A \cdot B) = \begin{pmatrix}
ae + bg \\
cf + dh
\end{pmatrix}
$$

Haciendo el producto elemento a elemento entre $A$ y $B^T$ y sumando:
$$
B^T = \begin{pmatrix}
e & g \\
f & h
\end{pmatrix}
$$

$$
A \odot B^T = \begin{pmatrix}
a & b \\
c & d
\end{pmatrix}
\odot
\begin{pmatrix}
e & g \\
f & h
\end{pmatrix}
=
\begin{pmatrix}
a \cdot e & b \cdot g \\
c \cdot f & d \cdot h
\end{pmatrix}
$$

$$
\text{np.sum}(A \odot B^T, \text{axis}=1) = \begin{pmatrix}
ae + bg \\
cf + dh
\end{pmatrix}
$$
$$
\text{diag}(A \cdot B) = \begin{pmatrix}
ae + bg \\
cf + dh
\end{pmatrix}
= \text{np.sum}(A \odot B^T, \text{axis}=1)
$$


In [ ]:
# Demostración en código:

# Se definen dimensiones arbitrarias
n = 5
k = 7

# Se crean matrices aleatorias, con esas dimensiones
A = np.random.randn(n, k)
B = np.random.randn(k, n)

# Forma costosa, con producto matricial completo y extrayendo diagonal
producto_matricial = A @ B  # Dimensión (n, n)
diagonal_original = np.diag(producto_matricial)

# Forma optimizada, con las sumas del producto elemento a elemento
# Se transpone B: (k, n) -> (n, k), para que coincida con A
optimizada = np.sum(A * B.T, axis=1)

print("Diagonal original:\n", diagonal_original)
print("Suma optimizada:\n", optimizada)
print("Son iguales?",np.allclose(diagonal_original, optimizada)) Se usa allclose por si hay errores de redondeo

Diagonal original:
 [ 0.85766051 -2.13315315 -0.50185242  1.93709816 -2.48132112]
Suma optimizada:
 [ 0.85766051 -2.13315315 -0.50185242  1.93709816 -2.48132112]
¿Son iguales? True


----
6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.

In [24]:
class EfficientQDA(TensorizedQDA):
    """
    Versión que evita la matriz nxn
    usando el producto de Hadamard para calcular solo la diagonal.
    """

    def _predict_log_conditionals_batch(self, X):
        # Dimensiones iniciales
        k, p, _ = self.tensor_means.shape  # self.tensor_means: (k, p, 1) → k clases, p features
        n = X.shape[1]                     # X: (p, n)                    → p features, n observaciones

        # Cálculo resta: D = X - medias
        D = X[None, :, :] - self.tensor_means   # X: (1, p, n) - Medias: (k, p, 1) → D: (k, p, n)

        # Cálculo de la distancia cuadrática 
        quad_terms = np.empty((k, n))  # Almacena distancias por cada k clase y observación n

        for j in range(k):
            D_j = D[j]                          # Shape: (p, n)
            inv_cov_j = self.tensor_inv_cov[j]  # Shape: (p, p)

            # ---------------------------------------------------------
            # MÉTODO EFICIENTE: usando producto de Hadamard
            # ---------------------------------------------------------
            # En vez de hacerr Q_j = D_j.T @ inv_cov_j @ D_j  → (n, n)
            # Calculamos lo siguiente diag(A @ B) = sum(A ⊙ B.T, axis=1)
            
            # Es inevitable tener que calcular alguno de los productos matriciales al haber 3 matrices en la multiplicación.
            # 1) A = D_j.T @ inv_cov_j  → (n, p) 
            A = D_j.T @ inv_cov_j
            
            # 2) B = D_j  → (p, n), entonces B.T = D_j.T → (n, p)
            # 3) diag(A @ D_j) = sum(A ⊙ D_j.T, axis=1)
            quad_terms[j, :] = np.sum(A * D_j.T, axis=1)
            # ---------------------------------------------------------

        # log_det es (k,). Lo convertimos a (k, 1) para operar con quad_terms
        log_det = np.log(LA.det(self.tensor_inv_cov))[:, None]
        
        # Fórmula final: 0.5 * log_det - 0.5 * distancia
        return 0.5 * log_det - 0.5 * quad_terms

    def predict(self, X):
        # Traigo los log-condicionales 
        log_cond = self._predict_log_conditionals_batch(X)

        # Sumamos el log_a_priori 
        log_post = self.log_a_priori[:, None] + log_cond

        # Elegimos la clase ganadora → por columna
        y_hat = np.argmax(log_post, axis=0)

        return y_hat.reshape(1, -1)

----
7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

In [ ]:
# FasterQDA
b.bench(FasterQDA)

# EfficientQDA
b.bench(EfficientQDA)

FasterQDA (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

In [11]:
df_summary = b.summary(baseline="QDA")
df_summary

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
QDA,8.16250,7.412496,1729.55460,1612.031217,0.886117,0.270096,0.002008,0.099219,0.001183,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,7.26470,0.616768,289.93255,3.242220,0.885303,0.268463,0.002143,0.154099,0.000018,1.123584,5.965369,1.006082,0.643866
FasterQDA,7.49610,1.225837,1095.82545,34.262841,0.884827,0.268951,0.001919,258.234673,0.000541,1.088900,1.578312,1.004255,0.000384
EfficientQDA,7.14195,0.627668,11.95610,0.908321,0.884890,0.269440,0.002171,15.743172,0.000000,1.142895,144.658760,1.002435,0.006302
EfficientQDA_2,7.14635,1.065439,16.78935,1.380324,0.884770,0.268402,0.002159,15.254768,0.000008,1.142191,103.014983,1.006310,0.006504


Vemos que TensorizedQDA entrena en el mismo tiempo que el QDA base y clasifica con la misma accuracy, pero es mucho más rápido en predicción.
FasterQDA no lo mejora porque empieza a comparar todas las observaciones contra todas y arma una matriz gigante N×N que consume 258 MB de RAM y ralentiza todo el proceso.

Se puede ver que EfficientQDA es el más rápido de todos para predecir y reduce enormemente el consumo de memoria respecto al FasterQDA, debido a que evita la matriz de $n \times n$, usando el producto elemento a elemento y la suma.

---------------------------------------------------------------------

## Cholesky

Hasta ahora todos los esfuerzos fueron enfocados en realizar una predicción más rápida. Los tiempos de entrenamiento (teóricos al menos) siguen siendo los mismos o hasta (minúsculamente) peores, dado que todas las mejoras siguen llamando al método `_fit_params` original de `QDA`.

La descomposición/factorización de [Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition#Statement) permite factorizar una matriz definida positiva $A = LL^T$ donde $L$ es una matriz triangular inferior. En particular, si bien se asume que $p \ll n$, invertir la matriz de covarianzas $\Sigma$ para cada clase impone un cuello de botella que podría alivianarse. Teniendo en cuenta que las matrices de covarianza son simétricas y salvo degeneración, definidas positivas, Cholesky como mínimo debería permitir invertir la matriz más rápido.

*Nota: observar que calcular* $A^{-1}b$ *equivale a resolver el sistema* $Ax=b$.

### 3) Diferencias entre implementaciones de `QDA_Chol`

8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?

Al aplicar la factorización de Cholesky a la matriz de convarianzas (matriz simétrica y definida positiva siempre que no hayan features perfectaemente colineales) se puede representar como $\Sigma = $LL^T$, de tal manera que la expresión $(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)$ se puede reescribir como:

$$
(x-\mu_j)^T (L_j L_j^T)^{-1} (x- \mu_j)
$$
y operando un poco:
$$
(x-\mu_j)^T (L_j^{-1})^T L_j^{-1} (x- \mu_j)
$$
$$
[L_j^{-1} (x- \mu_j)]^T [L_j^{-1} (x- \mu_j)]
$$
Si representamos $L_j^{-1} (x- \mu_j)$ como $y$, la expresión puede verse como
$$
y^Ty = ||y||^2
$$
donde $y$ puede calcularse resolviendo un sistema de ecuaciones lineales triangular, que es relativamente sencillo con sustitución hacia adelante:
$$
L_j y = x- \mu_j
$$
Por otro lado, respecto al logaritmo del determinante la matriz de covarianzas, se puede operar de la siguiente manera:
$$
\log |\Sigma_j| = \log |L_j L_j^T| = \log |L_j| + \log |L_j^T| = 2 \log |L_j|
$$
donde $|L_j|$, al ser $L_j$ una matriz triangular inferior, es el producto de sus elementos en la diagonal.

Finalmente, dependiendo de la implementación elegida, la forma cuadrática de QDA se puede reescribir como:

**Usando $L$** (QDA_Chol2):
$$\log{f_j(x)} = -\log{ \left( \prod_{i=1}^p l_{ii} \right)} - \frac{1}{2} \|y\|^2$$
donde $l_{ii}$ son los elementos diagonales de $L_j$

**Usando $L^{-1}$** (QDA_Chol1, QDA_Chol3):
$$\log{f_j(x)} = \log{ \left( \prod_{i=1}^p (L^{-1}_j)_{ii} \right)} - \frac{1}{2} \|y\|^2$$
donde $(L^{-1}_j)_{ii}$ son los elementos diagonales de $L^{-1}_j$

En resumen, aplicar Cholesky en QDA transforma un problema con  inversión de matrices en un problema de resolución de sistemas de ecuaciones lineales triangulares.

----
7. Explicar las diferencias entre `QDA_Chol1`y `QDA` y cómo `QDA_Chol1` llega, paso a paso, hasta las predicciones.

`QDA` en su método `_fit_params` calcula e invierte la matriz de covarianzas para cada clase y las almacena en `self.inv_covs`.

`QDA_Chol1`, en el mismo método, calcula la matriz de covarianzas y le aplica la factorización de Cholesky, obteniendo la matriz $L$. Luego, calcula la inversa de $L$ y la almacena en `self.L_invs`. Esto es para cada clase.

A la hora de realizar una predición, en el método `_predict_log_conditional`, en `QDA` prácticamente utiliza la formulación provista inicialmente para el cálculo de la log-verosimilitud, con algunos cambios.
$$
\log{f_j(x)} = \frac{1}{2}\log |\Sigma_j^{-1}| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)
$$
, mientras que `QDA_Chol1` aprovecha, por un lado, la propiedad de que el determinante de $L^{-1}$ es el producto de los elementos de su diagonal y, por otro, la equivalencia $$(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) = \|y\|^2$$, tal que el cálculo de la log-verosimilitud resulta:
$$
\log{f_j(x)} = \log{ \left( \prod_{i=1}^p (L^{-1}_j)_{ii} \right)} - \frac{1}{2} \|y\|^2
$$
donde $(L^{-1}_j)_{ii}$ son los elementos diagonales de $L^{-1}_j$ (la clase almacena `L_invs`, no `Ls`), y así llegando ambos al mismo resultado porque todos las sustituciones corresponden a equivalencias matemáticas, sin utilizar aproximaciones.

8. ¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?

`QDA_Chol1` y `QDA_Chol2` se diferencian en que la primera calcula la inversa de $L$ y luego, para predecir, utiliza el producto de esta matriz con $x$ insesgado para calcular $y$, mientras que la segunda se queda directamente con $L$ y, para predecir, resuelve el SEL triangular para obtener $y$.

`QDA_Chol3` es similar a `QDA_Chol1`, pero utiliza otra función para calcular $L^{-1}$: `QDA_Chol1` utiliza `numpy.inv()` y `QDA_Chol3` utiliza `scipy.linalg.lapack.dtrtri()`.

9. Comparar la performance de las 7 variantes de QDA implementadas hasta ahora ¿Qué se observa?¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás?¿Alguna que sea peor?

In [12]:
# Antes debe haberse ejecutado el bench de los otros modelos
b.bench(QDA_Chol1)

b.bench(QDA_Chol2)

b.bench(QDA_Chol3)

QDA_Chol1 (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

QDA_Chol1 (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

QDA_Chol2 (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

QDA_Chol2 (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

QDA_Chol3 (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

QDA_Chol3 (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
df_summary = b.summary(baseline="QDA")
df_summary

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
QDA,8.16250,7.412496,1729.55460,1612.031217,0.886117,0.270096,0.002008,0.099219,0.001183,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,7.26470,0.616768,289.93255,3.242220,0.885303,0.268463,0.002143,0.154099,0.000018,1.123584,5.965369,1.006082,0.643866
FasterQDA,7.49610,1.225837,1095.82545,34.262841,0.884827,0.268951,0.001919,258.234673,0.000541,1.088900,1.578312,1.004255,0.000384
EfficientQDA,7.14195,0.627668,11.95610,0.908321,0.884890,0.269440,0.002171,15.743172,0.000000,1.142895,144.658760,1.002435,0.006302
EfficientQDA_2,7.14635,1.065439,16.78935,1.380324,0.884770,0.268402,0.002159,15.254768,0.000008,1.142191,103.014983,1.006310,0.006504
QDA_Chol1,7.60500,1.187648,948.65465,37.157486,0.885433,0.269051,0.001967,0.094734,0.000349,1.073307,1.823166,1.003885,1.047339
QDA_Chol2,6.75825,0.586161,2340.74140,60.906744,0.885807,0.268723,0.001848,0.095211,0.000510,1.207783,0.738892,1.005110,1.042094
QDA_Chol3,7.25765,1.120846,947.43145,24.024118,0.884995,0.268723,0.002190,0.094589,0.000429,1.124675,1.825520,1.005110,1.048944


`QDA_Chol1` parece ser la mejor de las implementaciones de Cholesky en cuanto a velocidad en train y test, superando al `QDA` base pero sin hacerlo respecto de `EfficientQDA`. `QDA_Chol2` incluso resulta ser más lento para predecir y `QDA_Chol3` es bastante parecida a la primera.

Respecto a memoria, los 3 modelos con Cholesky tienen una leve mejora pero no parece significativa.

### 4) Optimización

12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.

In [18]:
class TensorizedChol(QDA_Chol3):
    def _fit_params(self, X, y):
        super()._fit_params(X, y)

        self.tensor_L_invs = np.stack(self.L_invs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self, x):
        unbiased_x = x - self.tensor_means
        y = self.tensor_L_invs @ unbiased_x

        return np.log(self.tensor_L_invs.diagonal(axis1=1, axis2=2).prod(axis=1)) - 0.5 * (y**2).sum(axis=1).flatten()

    def _predict_one(self, x):
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))


13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.

In [19]:
class EfficientChol(TensorizedChol):
    def _predict_log_conditionals_batch(self, X):
        unbiased_X = X[None, :, :] - self.tensor_means

        Y = self.tensor_L_invs @ unbiased_X

        log_det_term = np.log(self.tensor_L_invs.diagonal(axis1=1, axis2=2).prod(axis=1))[:, None]
        quad_term = (Y**2).sum(axis=1)
        
        return log_det_term - 0.5 * quad_term

    def predict(self, X):
        log_conditionals = self._predict_log_conditionals_batch(X)
        log_posteriori = self.log_a_priori[:, None] + log_conditionals

        y_hat = np.argmax(log_posteriori, axis=0)

        return y_hat.reshape(1, -1)      

14. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

In [22]:
b.bench(TensorizedChol)

b.bench(EfficientChol)

TensorizedChol (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

TensorizedChol (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

EfficientChol (MEM):   0%|          | 0/30 [00:00<?, ?it/s]

EfficientChol (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [23]:
df_summary = b.summary(baseline="QDA")
df_summary

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
QDA,8.16250,7.412496,1729.55460,1612.031217,0.886117,0.270096,0.002008,0.099219,0.001183,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,7.26470,0.616768,289.93255,3.242220,0.885303,0.268463,0.002143,0.154099,0.000018,1.123584,5.965369,1.006082,0.643866
FasterQDA,7.49610,1.225837,1095.82545,34.262841,0.884827,0.268951,0.001919,258.234673,0.000541,1.088900,1.578312,1.004255,0.000384
EfficientQDA,7.14195,0.627668,11.95610,0.908321,0.884890,0.269440,0.002171,15.743172,0.000000,1.142895,144.658760,1.002435,0.006302
EfficientQDA_2,7.14635,1.065439,16.78935,1.380324,0.884770,0.268402,0.002159,15.254768,0.000008,1.142191,103.014983,1.006310,0.006504
QDA_Chol1,7.60500,1.187648,948.65465,37.157486,0.885433,0.269051,0.001967,0.094734,0.000349,1.073307,1.823166,1.003885,1.047339
QDA_Chol2,6.75825,0.586161,2340.74140,60.906744,0.885807,0.268723,0.001848,0.095211,0.000510,1.207783,0.738892,1.005110,1.042094
QDA_Chol3,7.25765,1.120846,947.43145,24.024118,0.884995,0.268723,0.002190,0.094589,0.000429,1.124675,1.825520,1.005110,1.048944
TensorizedChol,7.70565,0.661728,74.05735,2.135236,0.886000,0.268539,0.001642,0.157776,0.000201,1.059288,23.354260,1.005796,0.628859


TODO: Conclusión final (tentativo)

## Conclusiones finales

Se puede observar que `EfficientChol` combina las mejores optimizaciones:
- **Entrenamiento más rápido** que QDA base gracias a Cholesky
- **Predicción más rápida** al paralelizar sobre k clases Y n observaciones
- **Menor consumo de memoria** al evitar la matriz N×N

Este modelo representa la mejor implementación, eliminando todos los loops 
explícitos (tanto sobre clases como sobre observaciones) y aprovechando las 
ventajas numéricas de la descomposición de Cholesky.

Comparado con QDA base:
- Entrenamiento: [X]% más rápido
- Predicción: [Y]% más rápido  
- Memoria: [Z]% menos consumo